# Extracting montly Measles and Pertussis incidence data by state (1951-Present):
## 1996-2017:
Weekly incidence data from 1996 up to present are available from CDC WONDER's MMWR tables.<br>
CDC WONDER is a query system for the analysis of public health data.<br>
MMWR (Morbidity and Mortality Weekly Report) is based on weekly reports to CDC by state health departments.<br>
Extarcting data from MMWR could be challenging because of the format inconsistencies:
- URL links to tables do not have similar format.
- Pertussis incidence data from 2002 to 2005 are reported as cumulative numbers.
- Data files have typo, "Cumulative" has been misspelled as "Cummulative".<br>

## 1951-1995
After reaching out to CDC INFO, it was revealed that they do not have the data prior to 1996 and they suggested to contact states directly.

# Pertussis Incidence Report Extracted From CDC Wonder

## Scraping CDC WONDER to extract the data

In [25]:
import os
import openpyxl
import numpy as np
from lxml import html
import requests
import sqlite3
import csv
import pandas as pd
from io import StringIO
import urllib
from bs4 import BeautifulSoup
import re
from datetime import date,timedelta

Running the above cell revealed that:
\begin{itemize}
  \item Pertussis stats. is available for all the years from 1996 to 2017
  \item Measles stats. is only available as a separate link from 1996 to 2002
\end{itemize}

In [26]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'Upstate New York',
        'NYC': 'New York City',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

states_abbrv=list(states.keys())

In [27]:
def MMWRWeekRange(Year):
     ##Returns a panda PeriodIndex containing MMWR weeks of the given year
     ## MMWR week definition: https://wwwn.cdc.gov/nndss/document/MMWR_week_overview.pdf
     ## The first day of any MMWR week is Sunday. MMWR week numbering is sequential beginning with 1 and
     ## incrementing with each week to a maximum of 52 or 53. MMWR week #1 of an MMWR year is the first week of
     ## the year that has at least four days in the calendar year. For example, if January 1 occurs on a Sunday, Monday,
     ## Tuesday or Wednesday, the calendar week that includes January 1 would be MMWR week #1. If January 1
     ## occurs on a Thursday, Friday, or Saturday, the calendar week that includes January 1 would be the last MMWR
     ## week of the previous year (#52 or #53). Because of this rule, December 29, 30, and 31 could potentially fall into
     ## MMWR week #1 of the following MMWR year.
    start=date(Year,1,1)
    end=date(Year,12,31)
    Jan1st_of_year=start.weekday()
    Jan1st_of_next_year=date(Year+1,1,1).weekday()
    rng = pd.period_range(start,end,freq='W-SAT')
    if Jan1st_of_year not in [0,1,2,6]:
        rng=rng[1:]
    if Jan1st_of_next_year in [0,1,2]:
        rng=rng[0:-1]
    return rng

In [28]:
def HyperHunter(URL,String,Tag): ### Returns the hyperlink that contains 'String' on URL page  
    page = requests.get(URL)
    soup = BeautifulSoup(page.text, 'html.parser')
    tag_value=soup.find(Tag,text=re.compile(String))
    return tag_value 

In [29]:
def MMWRParser(Year,Week,Disease,keyword,link_number):  
### Returns a Pandas dataframe containing cummulative incidence data for each state 
        reporting_date=date(Year-1,12,31)+timedelta(days=Week*7)
        str_Week=str(Week)
        if Week<10:
            str_Week='0'+str_Week
        url='https://wonder.cdc.gov/nndss/nndss_weekly_tables_menu.asp?mmwr_year='+str(Year)+'&mmwr_week='+str_Week
        Page=requests.get(url)
        soup=BeautifulSoup(Page.text,'html.parser')
        links=soup.find_all(string=re.compile(keyword))
        Export_hyperlink=links[link_number].parent.find_next('a')
        Export_hyperlink='https://wonder.cdc.gov/nndss/'+Export_hyperlink.attrs['href']
        MMWR_txt = requests.get(Export_hyperlink, allow_redirects=True)
        if MMWR_txt.status_code>300:
            Export_hyperlink='https://wonder.cdc.gov/'+Export_hyperlink.attrs['href']
            MMWR_txt = requests.get(Export_hyperlink, allow_redirects=True)
        content=MMWR_txt.text
        c=content.split("tab delimited data:")
        headersString=c[0].split("column labels in same order that data fields appears in each record below:")
        headersString=headersString[1]
        headers=headersString.splitlines()
        headers=headers[2:-1]
        c=c[1].split('\r\n\r\n')
        t=pd.read_table(StringIO(c[0]),sep='\t',header=None,index_col=0)
        t=t.iloc[:,0:len(headers)]  ### to get rid of the extar NaN columns
        t = t[~t.index.duplicated(keep='first')]
        t.columns=headers
        idx=t.index
        reg=Disease+'.*Total.*cum.*'+str(Year)
        t=t.filter(regex=reg)
        t=t.T
        t.columns=idx
        t.index=[Week]
        t.index.name='Week'
        weeks_rng=MMWRWeekRange(Year)
        week_period=weeks_rng[Week-1]
        Week_Start=week_period.start_time.strftime('%Y-%m-%d')
        Week_end=week_period.end_time.strftime('%Y-%m-%d')
        t.insert(0,'Start', Week_Start, allow_duplicates=False)
        t.insert(1,'End', Week_end, allow_duplicates=False)
        t=t.replace('-',0)
        t=t.replace('U',np.nan)
        t=t.apply(pd.to_numeric,errors='ignore',downcast='float')
        return t

In [30]:
# Disease='Measles'
# keyword='Measles'
# link_number=0
# Disease='Measles'
# keyword='Measles'
# file_name='Measles Cumulative Incidence Data by State 1996-2017.xlsx'
# #writer = pd.ExcelWriter(file_name)
# writer = pd.ExcelWriter(file_name, engine='openpyxl')
# book = openpyxl.load_workbook(file_name)
# writer.book = book  
# for Year in range(2000,2003):
#     print(Year)
#     Week_1=MMWRParser(Year,1,Disease,keyword,link_number)
#     cum_table=Week_1.copy()
#     number_of_weeks=len(MMWRWeekRange(Year))
#     for Week in range(2,number_of_weeks+1):
#         print(Week)
#         temp=MMWRParser(Year,Week,Disease,keyword,link_number)
#         cum_table=pd.concat([cum_table, temp], axis=0)
#     link_number=0
#     cum_table.to_excel(writer,str(Year))
#     writer.save()
#     writer = pd.ExcelWriter(file_name, engine='openpyxl')
#     book = openpyxl.load_workbook(file_name)
#     writer.book = book  

# writer.save()
# writer.close()

# for Year in range(2007,2018):
#     print(Year)
#     Week_1=MMWRParser(Year,1,Disease,keyword,link_number)
#     cum_table=Week_1.copy()
#     number_of_weeks=len(MMWRWeekRange(Year))
#     for Week in range(2,number_of_weeks+1):
#         print(Week)
#         temp=MMWRParser(Year,Week,Disease,keyword,link_number)
#         cum_table=pd.concat([cum_table, temp], axis=0)
#     link_number=0
#     cum_table.to_excel(writer,str(Year))
#     writer.save()
#     writer = pd.ExcelWriter(file_name, engine='openpyxl')
#     book = openpyxl.load_workbook(file_name)
#     writer.book = book  

# writer.save()
# writer.close()

In [31]:
def Measles2007onward(Year,Week):
    str_Week=str(Week)
    if Week<10:
        str_Week='0'+str_Week
    ### start and end of the week
    weeks_rng=MMWRWeekRange(Year)
    week_period=weeks_rng[Week-1]
    Week_Start=week_period.start_time.strftime('%Y-%m-%d')
    Week_end=week_period.end_time.strftime('%Y-%m-%d')
    ######
    keyword='Infrequently'
    url='https://wonder.cdc.gov/nndss/nndss_weekly_tables_menu.asp?mmwr_year='+str(Year)+'&mmwr_week='+str_Week
    Page=requests.get(url)
    soup=BeautifulSoup(Page.text,'html.parser')
    Submit_hyperlink=soup.find('a',string=re.compile(keyword))
    Export_hyperlink='https://wonder.cdc.gov/nndss/'+Submit_hyperlink.find_next('a').attrs['href']
    MMWR_txt = requests.get(Export_hyperlink, allow_redirects=True)
    if MMWR_txt.status_code==500:   ### Week 13 Year=2007 does not return anything
        a = pd.DataFrame(np.nan, index=[Week], columns=[])
        a.index.name='Week'
        a.insert(0,'Start', Week_Start, allow_duplicates=False)
        a.insert(1,'End', Week_end, allow_duplicates=False)
        return a
    elif MMWR_txt.status_code>300:   #### For Year>=2017, format of the export link changes
        txt_file_name=str(Year)+'-'+str_Week+'-table1.txt'
        Export_hyperlink='https://wonder.cdc.gov/nndss/static/'+str(Year)+'/'+str_Week+'/'+txt_file_name
        MMWR_txt = requests.get(Export_hyperlink, allow_redirects=True)
    content=MMWR_txt.text
    c=content.split("tab delimited data:")
    headersString=c[0].split("column labels in same order that data fields appears in each record below:")
    headersString=headersString[1]
    headers=headersString.splitlines()
    headers=headers[2:-1]
    c=c[1].split('\r\n\r\n')
    t=pd.read_table(StringIO(c[0]),sep='\t',header=None,index_col=0)
    t=t.iloc[:,0:len(headers)]  ### to get rid of the extar NaN columns
    t = t[~t.index.duplicated(keep='first')]
    t.columns=headers
    t=t.filter(regex='Measles',axis=0)
    t=t.filter(regex='States',axis=1)
    a=t.iloc[0].str[0:].str.extractall('(\D+)(\d+)[^)]')
    a.reset_index(drop=True,inplace=True)
    #a[0]=a[0].str.strip('^(^)\s^,')
    a[0]=a[0].str.replace('[\W]+','')
    cols=a[0]
    a=a.T
    a.columns=cols
    a=a.drop([0])
    a.index=[Week]
    a.index.name='Week'
    a.insert(0,'Start', Week_Start, allow_duplicates=False)
    a.insert(1,'End', Week_end, allow_duplicates=False)
    a=a.replace('-',0)
    a=a.replace('U',np.nan)
    a=a.apply(pd.to_numeric,errors='ignore',downcast='float')
    return a

# Scrapinp Years 2007-2017

In [33]:
file_name='Measles Weekly Incidence.xlsx'
if os.path.exists(file_name):
     writer = pd.ExcelWriter(file_name, engine='openpyxl')
     book = openpyxl.load_workbook(file_name)
     writer.book = book
else:
     writer = pd.ExcelWriter(file_name)
     

Years=range(2007,2018)  ### Years range for the years containg current week data
for Year in Years:   
    print(Year)
    Week_1=Measles2007onward(Year,1)
    current_table=Week_1.copy()
    number_of_weeks=len(MMWRWeekRange(Year))
    for Week in range(2,number_of_weeks+1):
        print(Week)
        temp=Measles2007onward(Year,Week)
        current_table=pd.concat([current_table, temp], axis=0)
    current_table.insert(0, 'Start',current_table.pop('Start'))            
    current_table.insert(1, 'End',current_table.pop('End'))
    current_table.rename(columns=states,inplace=True)
    current_table.to_excel(writer,str(Year))
    writer.save()
    writer = pd.ExcelWriter(file_name, engine='openpyxl')
    book = openpyxl.load_workbook(file_name)
    writer.book = book  

writer.save()
writer.close()


2012
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
2013
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
2014
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
2015
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
2016
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
2017
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
